In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

import ipywidgets as widgets
from IPython.display import display
from datetime import date, timedelta

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'notebook'

import ptm_lib as ptm
%load_ext autoreload
%autoreload 2

# import pandas_datareader as pdr
# from pandas_datareader import wb
# import quandl
# import matplotlib.pyplot as plt
# from matplotlib import colors

In [7]:
def loadNmerge_ticker(data, win_size, ticker, from_date, to_date, interval, pos):
    """ Function that takes as input a DataFrame (data), loads information for the ticker, merges it to the DataFrame
        and calculates the YoY of each column. The rolling correlation of size (win_size) is calculated between the
        YoY ticker vs the YoY of the column indicated by pos from the unmerged dataframe.                           
    """
    periods_to_year = {'d':260,'w':52,'m':12,'q':3, '1d':260,'1wk':52,'1mo':12,'3mo':4 }
    if interval not in periods_to_year:
        raise ValueError(f"Invalid interval: {interval}. Allowed values: {list(periods_to_year.keys())}")
    df = yf.download(ticker, start=from_date, end=to_date, interval=interval, auto_adjust=False)
    df = df[['Adj Close']].copy()
    df.rename(columns={'Adj Close':ticker}, inplace=True)
    data = pd.merge(data, df, how='outer', left_index=True, right_index=True)
    data = data.dropna()
    
    for col in data.columns:
        data[col+' YoY'] = ptm.returns(data, col, periods_to_year[interval]) # shifted by 4 quarters (1 year)        
    data = data.dropna()

    return data


def corr_lag_plot(data, win_size, ticker):
    """ Function that takes as input a DataFrame (df) and plots the correlation of tick_compare against 
        the ticker lagged returns       
    """
    # First Figure - Lagging Correlation
    fig = go.Figure()
    tick_compare = data.columns[1]
    print('\nAverage 10-yr Correlation:'+tick_compare+' YoY vs '+ticker+' YoY')
    lags = ['no-lag','3mo-lag','6mo-lag','9mo-lag','12mo-lag']
    for i,val in enumerate(lags):
        df = data[tick_compare+' YoY'].rolling(window=win_size).corr(data[ticker+' YoY'].shift(i)).dropna()
        print(val+': \t'+str(df.mean()))
        fig.add_trace(go.Scatter(x=df.index, y=df, mode='lines', name=val))
    fig.update_layout(title_text="<b>"+ tick_compare +' YoY vs '+ticker+" YoY Correlation<b>", title_x=0.5)    
    fig.update_xaxes(title_text="<b>Dates<b>")
    fig.update_yaxes(title_text="<b>Correlation</b>")
    fig.show()
    
    # Second Figure - Ticker Compare vs Ticker
    fig = make_subplots(specs=[[{"secondary_y": True}]])   # Create figure with secondary y-axis
    fig.add_trace(go.Scatter(x=data.index, y=data[tick_compare+' YoY'], name=tick_compare+" YoY"), secondary_y=False)
    fig.add_trace(go.Scatter(x=data.index, y=data[ticker+' YoY'], name=ticker+' YoY'), secondary_y=True)
    fig.update_layout(title_text="<b>"+tick_compare+" vs "+ticker+"<b>", title_x=0.5) # Add figure title
    fig.update_xaxes(title_text="<b>Dates<b>")                                        # Set x-axis title
    fig.update_yaxes(title_text="<b>"+tick_compare+" YoY</b>", secondary_y=False)     # Set y-axes titles
    fig.update_yaxes(title_text="<b>"+ticker+" YoY</b>", secondary_y=True)
    fig.show()
    

In [8]:
from_date = date(1928, 1, 1) 
today = date.today()

tickers = ['GDP','GDPC1'] # Gross Domestic Product for United States
position = 1              # index position of Read GDP
file = 'GDP_US'
interval = 'q' 
data = ptm.loadNupdate(tickers, from_date, today, interval, file)

ticker = '^GSPC'
window = 40               # window in interval(quarters) e.g., 40 (10 year)
interval="3mo"
data = loadNmerge_ticker(data, window, ticker, from_date, today, interval, position)
corr_lag_plot(data, window, ticker)

Next Timestamp:  2024-04-09 00:00:00
Today Timestamp: 2025-03-11 00:00:00


[*********************100%***********************]  1 of 1 completed


 Dataset updated and saved 



Price,Adj Close,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,,
1985-01-01,180.660004,180.660004,183.949997,163.360001,167.199997,7021420000
1985-04-01,191.850006,191.850006,191.850006,177.860001,180.660004,6449220000
1985-07-01,182.080002,182.080002,196.070007,179.449997,191.850006,6246430000
1985-10-01,211.279999,211.279999,213.080002,181.160004,182.059998,7792990000
1986-01-01,238.899994,238.899994,240.110001,202.600006,211.279999,8992710000
...,...,...,...,...,...,...
2024-01-01,5254.350098,5254.350098,5264.850098,4682.109863,4745.200195,250104320000
2024-04-01,5460.479980,5460.479980,5523.640137,4953.560059,5257.970215,244622510000


yes it works
test
again


Price,Adj Close
Ticker,^GSPC
Date,
1985-01-01,180.660004
1985-04-01,191.850006
1985-07-01,182.080002
1985-10-01,211.279999
1986-01-01,238.899994
...,...
2024-01-01,5254.350098
2024-04-01,5460.479980


MultiIndex([('Adj Close', '^GSPC'),
            (    'Close', '^GSPC'),
            (     'High', '^GSPC'),
            (      'Low', '^GSPC'),
            (     'Open', '^GSPC'),
            (   'Volume', '^GSPC')],
           names=['Price', 'Ticker'])


,GDP,GDPC1
date,,
1947-01-01,243.164,2182.681
1947-04-01,245.968,2176.892
1947-07-01,249.585,2172.432
1947-10-01,259.745,2206.452
1948-01-01,265.742,2239.682


MergeError: Not allowed to merge between different levels. (1 levels on the left, 2 on the right)

In [ ]:
# from_date = date(1928, 1, 1) 
# today = date.today()
# # Gross Domestic Product for United Kingdom
# tickers = ['UKNGDP','NGDPRSAXDCGBQ']
# file = 'GDP_UK'
# frequency = 'q' # quarterly

# data = ptm.loadNupdate(tickers, from_date, today, frequency, file)
# print(data)

# # FTSE - FTSE 100: Comprises the 100 largest companies listed on the London Stock Exchange (LSE). It's a key benchmark for the UK stock market.
# # FTMC - FTSE 250: Tracks the 250 next largest companies after the FTSE 100 on the LSE, representing mid-cap stocks.
# # FTLC - FTSE 350: Combines the FTSE 100 and FTSE 250, offering a broader representation of the UK stock market.
# ticker = '^FTSE'

# win_size = 40 # window in quarters e.g., 40 (10 year)
# data = loadNmerge_ticker(data, win_size, ticker, from_date, today)
# print(data)
# corr_lag_plot(data, ticker)

In [ ]:
# from_date = date(1928, 1, 1) 
# today = date.today()
# # Gross Domestic Product for European Union (27 Countries from 2020)
# tickers = ['CPMNACSCAB1GQEU272020','CLVMNACSCAB1GQEU272020']
# file = 'GDP_EU'
# frequency = 'q' # quarterly

# data = ptm.loadNupdate(tickers, from_date, today, frequency, file)
# print(data)

# # FTSE - FTSE 100: Comprises the 100 largest companies listed on the London Stock Exchange (LSE). It's a key benchmark for the UK stock market.
# # FTMC - FTSE 250: Tracks the 250 next largest companies after the FTSE 100 on the LSE, representing mid-cap stocks.
# # FTLC - FTSE 350: Combines the FTSE 100 and FTSE 250, offering a broader representation of the UK stock market.
# ticker = '^STOXX'

# win_size = 40 # window in quarters e.g., 40 (10 year)
# data = loadNmerge_ticker(data, win_size, ticker, from_date, today)
# print(data)
# corr_lag_plot(data, ticker)

In [ ]:
# from_date = date(1928, 1, 1) 
# today = date.today()
# # Gross Domestic Product for European Union (27 Countries from 2020)
# tickers = ['EUNNGDP','CLVMEURSCAB1GQEA19']
# file = 'GDP_EZ'
# frequency = 'q' # quarterly

# data = ptm.loadNupdate(tickers, from_date, today, frequency, file)
# print(data)

# # FTSE - FTSE 100: Comprises the 100 largest companies listed on the London Stock Exchange (LSE). It's a key benchmark for the UK stock market.
# # FTMC - FTSE 250: Tracks the 250 next largest companies after the FTSE 100 on the LSE, representing mid-cap stocks.
# # FTLC - FTSE 350: Combines the FTSE 100 and FTSE 250, offering a broader representation of the UK stock market.
# ticker = '^STOXX'

# win_size = 40 # window in quarters e.g., 40 (10 year)
# data = loadNmerge_ticker(data, win_size, ticker, from_date, today)
# print(data)
# corr_lag_plot(data, ticker)